In [1]:
# Set up the environment
from importlib.machinery import WindowsRegistryFinder
import os

#Change the current working directory: insert directory here

os.chdir('/home/koplenig/anderes/Cassandra')
os.getcwd()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
import pickle as pickle
from pmdarima.arima import ndiffs
from sklearn.metrics import mean_squared_error
#https://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error
from pmdarima.metrics import smape
%matplotlib inline

import pmdarima as pm
print(f"Using pmdarima {pm.__version__}")
# Using pmdarima 1.5.2

# We follow the example presented here https://alkaline-ml.com/pmdarima/usecases/stocks.html
# https://github.com/alkaline-ml/pmdarima
# To install: pip install pmdarima

# Samples per corpus were prepared with Stata (prepare_raw.do & prepare_samples.do --> csv file for each corpus)

#Define function to calculate the mean absolute percentage error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Hyndman on sMAPE: https://robjhyndman.com/hyndsight/smape/

# Define function to calculate the Tofallis accuracy measure (log (predicted/actual))

def acc_ratio(y_true, y_pred):
    # Calculate the sum of log(y_pred/y_true)^2
    return np.sum(np.log(y_pred/y_true)**2)
    


Using pmdarima 1.8.4


In [5]:
# In this variant, we predict the whole test set at once for the full-sample forecast.
# Results dataframe: corpus, first_month, last_month, gaps, train_len, test_len, auto.order, intercept mean_squared_error
df_results = pd.DataFrame(columns=['corpus', 'type' , 'forecasttype', 'first_month', 'last_month', 'train_len', 'test_len', 'auto', 'smape', 'mape', 'acc_ratio' ])
corpora = ["bloggmix", "da", "familjeliv", "flashback", "gp", "svt", "twitter","all", "synthetic"]
# Two different models: one with exogenous features, one without
types = ["exog", "no_exog"]

# Two different forecast types: one-step-ahead and full-sample
forecasttypes = ["one_step", "full_sample"]



# Loop over corpora, types and types of forecasting (one-step-ahead vs full-sample)
for corpus in corpora:
    for type in types:
        for forecasttype in forecasttypes:
            print(f"Processing {corpus} with {type}")
            # Read in the data
            df = pd.read_csv(f"samples/{corpus}_monthly_sample.csv", delimiter='\t', header=0, index_col=False)

            # Split the data into train and test according to variable test
            train_data = df[df['test'] == 0]
            test_data = df[df['test'] == 1]

            # Extract first and last month
            first_month = train_data['month'].iloc[0]
            last_month = train_data['month'].iloc[-1]

            # Compute train and test length
            train_len = len(train_data)
            test_len = len(test_data)


            if type == "exog":
                # Exogenous features
                exogenous_features = ['subject', 'voice', 'att_before', 'att_after', 'att_interaction', 'distance_to_att_words', 'inf_length', 'attraction']
                # Fit the model
                model = pm.auto_arima(train_data['outcome'], X=train_data[exogenous_features], seasonal=False, suppress_warnings=True, stepwise=False, maxiter=50)
                model.fit(train_data['outcome'], X=train_data[exogenous_features])
                # Loop over all test observations
                forecasts = []
                # For 'one-step-ahead' forecasting, we loop over all test observations
                if forecasttype == "one_step":
                    for i in range(len(test_data)):
                        # Forecast the next 1 values
                        forecast = model.predict(n_periods=1, X=test_data[exogenous_features].iloc[i].values.reshape(1, -1))
                        # Update the model with the new observation
                        model.update(test_data['outcome'].iloc[i], X=test_data[exogenous_features].iloc[i].values.reshape(1, -1), maxiter=50)
                        # Append the forecast to forecasts
                        forecasts.append(forecast[0])
                # For 'full-sample' forecasting, we forecast the whole test set at once
                else:
                   forecasts = model.predict(n_periods=len(test_data), X=test_data[exogenous_features])         
            else:
                # Fit the model
                model = pm.auto_arima(train_data['outcome'], seasonal=False, suppress_warnings=True, stepwise=True)
                model.fit(train_data['outcome'])
                # Loop over all test observations
                forecasts = []
                # For 'one-step-ahead' forecasting, we loop over all test observations
                if forecasttype == "one_step":
                    for i in range(len(test_data)):
                        # Forecast the next 1 values
                        forecast = model.predict(n_periods=1)
                        # Update the model with the new observation
                        model.update(test_data['outcome'].iloc[i], maxiter=50)
                        # Append the forecast to forecasts
                        forecasts.append(forecast[0])
                # For 'full-sample' forecasting, we forecast the whole test set at once
                else:
                   forecasts = model.predict(n_periods=len(test_data))         
            # Print ARIMA order
            print(f"ARIMA order: {model.order}")
            # Print model
            print(f"Model: {model}")
            # Print model summary
            #print(model.summary())

        
            # Save results
            df_results = df_results.append(
                {'corpus': corpus, 'type': type, 'forecasttype': forecasttype,  'first_month': first_month, 'last_month': last_month, 
                'train_len': train_len, 'test_len': test_len, 'auto': model, 
                'smape': smape(test_data['outcome'], forecasts), 
                'mape': mean_absolute_percentage_error(test_data['outcome'], forecasts), 
                'acc_ratio': acc_ratio(test_data['outcome'], forecasts)}, ignore_index=True) 
            

            # Export results
            df_results.to_csv("results.csv", sep='\t', index=False)

            # Add forecast to test data
            test_data['forecast'] = forecasts

            # Add dummy column to train_data
            train_data['forecast'] = np.nan

            # Concatenate train and test data
            df = pd.concat([train_data, test_data])

            # Export data and name it according to corpus and model
            df.to_csv(f"samples/{corpus}_{type}_{forecasttype}_ARIMA.csv", sep='\t', index=False)
            

# Replace acc_ratio = sqrt(acc_ratio)*100/test_len
df_results['acc_ratio'] = np.sqrt(df_results['acc_ratio'])*100/df_results['test_len']

# Sort results by corpus and forecast type
df_results = df_results.sort_values(by=['corpus', 'forecasttype'])
df_results

Processing bloggmix with exog
ARIMA order: (0, 1, 1)
Model:  ARIMA(0,1,1)(0,0,0)[0] intercept
Processing bloggmix with exog


/home/koplenig/.local/lib/python3.6/site-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/koplenig/.local/lib/python3.6/site-packages/ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ARIMA order: (0, 1, 1)
Model:  ARIMA(0,1,1)(0,0,0)[0] intercept
Processing bloggmix with no_exog


/home/koplenig/.local/lib/python3.6/site-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/koplenig/.local/lib/python3.6/site-packages/ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ARIMA order: (0, 1, 1)
Model:  ARIMA(0,1,1)(0,0,0)[0] intercept
Processing bloggmix with no_exog


...

ARIMA order: (1, 2, 2)
Model:  ARIMA(1,2,2)(0,0,0)[0]          
Processing synthetic with no_exog


/home/koplenig/.local/lib/python3.6/site-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/koplenig/.local/lib/python3.6/site-packages/ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ARIMA order: (1, 2, 2)
Model:  ARIMA(1,2,2)(0,0,0)[0]          


/home/koplenig/.local/lib/python3.6/site-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/koplenig/.local/lib/python3.6/site-packages/ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,corpus,type,forecasttype,first_month,last_month,train_len,test_len,auto,smape,mape,acc_ratio
29,all,exog,full_sample,2004m1,2016m12,156,60,"ARIMA(0,1,1)(0,0,0)[0] intercept",7.765622,8.182416,1.14773
31,all,no_exog,full_sample,2004m1,2016m12,156,60,"ARIMA(0,2,3)(0,0,0)[0]",2.932024,2.933661,0.446501
28,all,exog,one_step,2004m1,2016m12,156,60,"ARIMA(0,1,1)(0,0,0)[0] intercept",1.431702,1.434138,0.240159
30,all,no_exog,one_step,2004m1,2016m12,156,60,"ARIMA(0,2,3)(0,0,0)[0]",1.566139,1.560361,0.26008
1,bloggmix,exog,full_sample,2005m1,2013m12,108,36,"ARIMA(0,1,1)(0,0,0)[0] intercept",9.336145,10.087494,1.99364
3,bloggmix,no_exog,full_sample,2005m1,2013m12,108,36,"ARIMA(0,1,1)(0,0,0)[0] intercept",14.783254,16.581787,2.97291
0,bloggmix,exog,one_step,2005m1,2013m12,108,36,"ARIMA(0,1,1)(0,0,0)[0] intercept",7.045096,7.421316,1.49634
2,bloggmix,no_exog,one_step,2005m1,2013m12,108,36,"ARIMA(0,1,1)(0,0,0)[0] intercept",4.864196,5.092958,1.11687
5,da,exog,full_sample,2013m1,2019m12,84,24,"ARIMA(0,1,4)(0,0,0)[0] intercept",31.462071,33.733541,9.17315
7,da,no_exog,full_sample,2013m1,2019m12,84,24,"ARIMA(1,1,0)(0,0,0)[0]",26.465954,25.560592,6.71123
